# **Representación basada en Bag of Words** 

Instalar librerías a través de la terminal.


In [1]:
#pip install spacy
#pip install nltk
#pip install sklearn
#sudo python3 -m spacy download es  <--- Mac
#spacy download es                  <--- Windows 10

Importar funciones necesarias.

In [2]:
import pandas as pd
import re
import spacy as spc
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

# Mostrar dataframes 
from IPython.display import display, HTML

## **Datos de entrada**

Generamos dos instancias, la primera obtenida de una [noticia deportiva](https://cnnespanol.cnn.com/2022/11/04/pique-retirada-barcelona-orix/) y la otra de una [noticia política](https://www.jornada.com.mx/notas/2023/09/16/politica/amlo-agradece-contribucion-fundamental-de-semar-y-sedena/).

In [3]:
oracion1 = "El defensa del Barcelona Gerard Piqué puso fin a su brillante carrera el jueves, al anunciar que se retirará del fútbol cuando LaLiga se interrumpa por el Mundial este mes."
oracion2 = "A través de sus redes sociales, el presidente Andrés Manuel López Obrador sostuvo que \' es permanente nuestro reconocimiento a las secretarías de la a Defensa Nacional y de Marina...\'."

print('Oración 1 (Nota deportiva):\n', oracion1)
print('\nOración 2 (Nota política):\n', oracion2)

Oración 1 (Nota deportiva):
 El defensa del Barcelona Gerard Piqué puso fin a su brillante carrera el jueves, al anunciar que se retirará del fútbol cuando LaLiga se interrumpa por el Mundial este mes.

Oración 2 (Nota política):
 A través de sus redes sociales, el presidente Andrés Manuel López Obrador sostuvo que ' es permanente nuestro reconocimiento a las secretarías de la a Defensa Nacional y de Marina...'.


## **Procesamiento**

### Limpieza de datos

Definimos una función para limpiar los datos de una oración:

In [4]:
def limpia_oracion(oracion):
    oracion = oracion.lower()
    oracion = re.sub(r"@\S+", "", oracion)  # Eliminar menciones a usuarios
    oracion = re.sub("http[s]?\://\S+", "", oracion)  # Eliminar enlaces
    oracion = re.sub(r"#\S+", "", oracion)  # Eliminar hashtags (No siempre es lo mejor)
    oracion = re.sub(r"[0-9]", "", oracion)  # Eliminar números
    oracion = re.sub(r"(\(.*\))|(\[.*\])", "", oracion)  # Eliminar paréntesis y corchetes
    oracion = re.sub(r"\n", "", oracion)  # Eliminar caracteres de nueva línea
    oracion = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", "", oracion)  # Eliminar varios patrones
    oracion = re.sub(r"(\.)|(,)", "", oracion)  # Eliminar puntos y comas
    oracion = re.sub(r"[¡!]", "", oracion)  # Eliminar signos de admiración 
    oracion = re.sub(r"[¿?]", "", oracion)  # Eliminar signos de exclamación
    print(oracion)

Limpiamos la oración 1 y 2 respectivamente.

In [5]:
display(limpia_oracion(oracion1))

el defensa del barcelona gerard piqué puso fin a su brillante carrera el jueves al anunciar que se retirará del fútbol cuando laliga se interrumpa por el mundial este mes


None

In [6]:
display(limpia_oracion(oracion2))

a través de sus redes sociales el presidente andrés manuel lópez obrador sostuvo que ' es permanente nuestro reconocimiento a las secretarías de la a defensa nacional y de marina'


None

### Tokenización

Tokenización oración 1:

In [7]:
tokens1 = word_tokenize(oracion1)
print(tokens1)

['El', 'defensa', 'del', 'Barcelona', 'Gerard', 'Piqué', 'puso', 'fin', 'a', 'su', 'brillante', 'carrera', 'el', 'jueves', ',', 'al', 'anunciar', 'que', 'se', 'retirará', 'del', 'fútbol', 'cuando', 'LaLiga', 'se', 'interrumpa', 'por', 'el', 'Mundial', 'este', 'mes', '.']


Tokenización oración 2:

In [8]:
tokens2 = word_tokenize(oracion2)
print(tokens2)

['A', 'través', 'de', 'sus', 'redes', 'sociales', ',', 'el', 'presidente', 'Andrés', 'Manuel', 'López', 'Obrador', 'sostuvo', 'que', "'", 'es', 'permanente', 'nuestro', 'reconocimiento', 'a', 'las', 'secretarías', 'de', 'la', 'a', 'Defensa', 'Nacional', 'y', 'de', 'Marina', '...', "'", '.']


### Stopwords

Definimos una función para eliminar _stopwords_.

In [9]:
def delete_stopword(tokens):
    spanish_stopwords = stopwords.words('spanish')
    palabras_filtradas = [palabra for palabra in tokens if palabra not in spanish_stopwords]
    return palabras_filtradas

Eliminamos las _stopwords_ para la oración 1 y 2 respectivamente.

In [10]:
palabras_filtradas1 = delete_stopword(tokens1)
print(palabras_filtradas1)

['El', 'defensa', 'Barcelona', 'Gerard', 'Piqué', 'puso', 'fin', 'brillante', 'carrera', 'jueves', ',', 'anunciar', 'retirará', 'fútbol', 'LaLiga', 'interrumpa', 'Mundial', 'mes', '.']


In [11]:
palabras_filtradas2 = delete_stopword(tokens2)
print(palabras_filtradas2)

['A', 'través', 'redes', 'sociales', ',', 'presidente', 'Andrés', 'Manuel', 'López', 'Obrador', 'sostuvo', "'", 'permanente', 'reconocimiento', 'secretarías', 'Defensa', 'Nacional', 'Marina', '...', "'", '.']


### Lematización

Definimos una función para lematizar una oración dada.

In [12]:
def lematiza_oracion(palabras_filtradas):
    nlp = spc.load("es_core_news_sm")
    lema = nlp(" ".join(palabras_filtradas))
    oracion_lematizada = " ".join([token.lemma_ for token in lema])
    return oracion_lematizada

Lematización de la oración 1.

In [13]:
oracion_lematizada1 = lematiza_oracion(palabras_filtradas1)
print(oracion_lematizada1)

el defensa Barcelona Gerard Piqué poner fin brillante carrera jueves , anunciar retirar fútbol LaLiga interrumpa Mundial mes .


Lematización de la oración 2.

In [14]:
oracion_lematizada2 = lematiza_oracion(palabras_filtradas2)
print(oracion_lematizada2)

a través red social , presidente Andrés Manuel López Obrador sostener ' permanente reconocimiento secretaría Defensa Nacional Marina ... ' .


## **Bag of Words**

Definimos una función para obtener los vectores _Bag of Words_ y el vocabulario de la oración lematizada.

In [15]:
def get_bow(oracion_lematizada):
    vectorizador = CountVectorizer()
    vectores = vectorizador.fit_transform([oracion_lematizada])
    vocabulario = vectorizador.get_feature_names_out()
    return vectores, vocabulario

Utilizamos la función anterior con las oraciones 1 y 2.

In [16]:
#Oración 1
vectores1, vocabulario1 = get_bow(oracion_lematizada1)

#Oración 2
vectores2, vocabulario2 = get_bow(oracion_lematizada2)

### Resultados

Oración 1.

In [17]:
print("Oración de entrada:", oracion1)
print("\nOración lematizada:", oracion_lematizada1)
print("\nVectores Bag of Words:", vectores1.toarray())
print("\nVocabulario:", vocabulario1)

Oración de entrada: El defensa del Barcelona Gerard Piqué puso fin a su brillante carrera el jueves, al anunciar que se retirará del fútbol cuando LaLiga se interrumpa por el Mundial este mes.

Oración lematizada: el defensa Barcelona Gerard Piqué poner fin brillante carrera jueves , anunciar retirar fútbol LaLiga interrumpa Mundial mes .

Vectores Bag of Words: [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]

Vocabulario: ['anunciar' 'barcelona' 'brillante' 'carrera' 'defensa' 'el' 'fin'
 'fútbol' 'gerard' 'interrumpa' 'jueves' 'laliga' 'mes' 'mundial' 'piqué'
 'poner' 'retirar']


Oración 2.

In [18]:
print("Oración de entrada:", oracion2)
print("\nOración lematizada:", oracion_lematizada2)
print("\nVectores Bag of Words:", vectores2.toarray())
print("\nVocabulario:", vocabulario2)

Oración de entrada: A través de sus redes sociales, el presidente Andrés Manuel López Obrador sostuvo que ' es permanente nuestro reconocimiento a las secretarías de la a Defensa Nacional y de Marina...'.

Oración lematizada: a través red social , presidente Andrés Manuel López Obrador sostener ' permanente reconocimiento secretaría Defensa Nacional Marina ... ' .

Vectores Bag of Words: [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]

Vocabulario: ['andrés' 'defensa' 'lópez' 'manuel' 'marina' 'nacional' 'obrador'
 'permanente' 'presidente' 'reconocimiento' 'red' 'secretaría' 'social'
 'sostener' 'través']


### Construir Dataframe

Empezamos por construir por separado los dataframes para la oración 1 y 2.

In [19]:
#Oración 1
df1_bw = pd.DataFrame.sparse.from_spmatrix(vectores1,columns = vocabulario1)
display(df1_bw.iloc[:,:10])  # Muestra las primeras 10 columnas del dataframe

,anunciar,barcelona,brillante,carrera,defensa,el,fin,fútbol,gerard,interrumpa
0,1,1,1,1,1,1,1,1,1,1


In [20]:
#Oración 2
df2_bw = pd.DataFrame.sparse.from_spmatrix(vectores2,columns = vocabulario2)
display(df2_bw.iloc[:,:10])  # Muestra las primeras 10 columnas del dataframe

,andrés,defensa,lópez,manuel,marina,nacional,obrador,permanente,presidente,reconocimiento
0,1,1,1,1,1,1,1,1,1,1


Agregamos una nueva columna a cada dataframe para clasificar la oración de acuerdo al tipo de noticia que corresponde.

In [21]:
#Oración 1
df1_bw['Clase'] = 'deporte'
display(df1_bw.iloc[:, -10:])   # Muestra las últimas 10 columnas del dataframe

,gerard,interrumpa,jueves,laliga,mes,mundial,piqué,poner,retirar,Clase
0,1,1,1,1,1,1,1,1,1,deporte


In [22]:
#Oración 2
df2_bw['Clase'] = 'política'
display(df2_bw.iloc[:, -10:])   # Muestra las últimas 10 columnas del dataframe

,obrador,permanente,presidente,reconocimiento,red,secretaría,social,sostener,través,Clase
0,1,1,1,1,1,1,1,1,1,política


Unimos los dataframes _df1_bw_ y _df2_bw_ en el nuevo dataframe _df_bw_.

In [23]:
df_bw = pd.merge(df1_bw, df2_bw, on='Clase', how='outer')
display(df_bw.iloc[:, -10:])   # Muestra las últimas 10 columnas del dataframe

,nacional,obrador,permanente,presidente,reconocimiento,red,secretaría,social,sostener,través
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Cambiamos los valores nulos por 0, ya que esto es equivalente a decir que esa palabra no aparece en la oración dada.

In [24]:
df_bw = df_bw.fillna(0)
display(df_bw.iloc[:, -10:])   # Muestra las últimas 10 columnas del dataframe

,nacional,obrador,permanente,presidente,reconocimiento,red,secretaría,social,sostener,través
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


La función _merge_ de Pandas, por defecto, cuando los dataframes tienen columnas con el mismo nombre (lo cual puede suceder en este caso cuando una palabra aparece en el vocabulario de ambas oraciones) al realizar la operación de "_merge_" o mezclado de los dataframes, crea dos columnas con el mismo nombre pero con sufijos '_ _x_' y '_ _y_' respectivamente. 

Para combinar estas columnas duplicadas, ejecutamos el siguiente código

In [25]:
# Obtener la lista de columnas duplicadas
columnas_duplicadas = [col for col in df_bw.columns if col.endswith(('_x', '_y'))]
columnas_duplicadas

# Unir las columnas repetidas en una sola columna
for col in columnas_duplicadas:
    col_original = col.replace('_x', '').replace('_y', '')
    df_bw[col_original] = df_bw[col_original + '_x'] + df_bw[col_original + '_y']
    
# Eliminar las columnas duplicadas 
df_bw.drop(columnas_duplicadas, axis=1, inplace=True)

Reorganizamos el dataframe para que la columna 'Clase' quede hasta el final del dataframe

In [26]:
column_order = [col for col in df_bw.columns if col != 'Clase'] + ['Clase']
df_bw = df_bw[column_order]

display(df_bw.iloc[:, -10:])   # Muestra las últimas 10 columnas del dataframe

,permanente,presidente,reconocimiento,red,secretaría,social,sostener,través,defensa,Clase
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,deporte
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,política


De esta manera, obtenemos la estructura de datos final, que es una representación basada en el modelo Bag of Words de dos oraciones, con el atributo de clase como su última columna.

### **Referencias**

1. CNN Español. (2022, noviembre 4). _Gerard Piqué anuncia su retirada del fútbol: hijos, carrera y títulos con el Barcelona, la selección de España y más_. CNN Español. https://cnnespanol.cnn.com/2022/11/04/pique-retirada-barcelona-orix/

2. La Jornada. (2023, septiembre 16). _Agradece AMLO “contribución fundamental” de Semar y Sedena en seguridad_. https://www.jornada.com.mx/notas/2023/09/16/politica/amlo-agradece-contribucion-fundamental-de-semar-y-sedena/